In [6]:
import pandas as pd

# colunas de interesse
selected_columns = ['Q3A', 'Q5A', 'Q10A', 'Q13A', 'Q16A', 'Q17A', 'Q21A', 'Q24A', 'Q26A', 'Q31A', 'Q34A', 'Q37A', 'Q38A', 'Q42A',
                    'gender', 'country', 'education', 'age', 'married']

file_path = 'originalDataset.csv'


df = pd.read_csv(file_path, delimiter='\t')


initial_row_count = len(df)
print(f"Número inicial de linhas no originalDataset: {initial_row_count}")


df.replace('NONE', pd.NA, inplace=True)

# eliminar linhas com nan
df.dropna(inplace=True)

#criterios
criteria = {
    'education': (1, 4),
    'gender': (1, 3),
    'married': (1, 3),
    'age': (18, 100)
}

for column, (low, high) in criteria.items():
    df = df[df[column].between(low, high)]

# contando participantes
country_counts = df['country'].value_counts()

# filtrar países com 30 ou mais participantes
countries_with_min_participants = country_counts[country_counts >= 30].index


df_filtered = df[df['country'].isin(countries_with_min_participants)]


filtered_row_count = len(df_filtered)
print(f"Número de linhas após todos os filtros: {filtered_row_count}")

# Selecionar apenas as colunas desejadas
df_filtered = df_filtered[selected_columns]

# -1 questões da depressoa
item_columns = ['Q3A', 'Q5A', 'Q10A', 'Q13A', 'Q16A', 'Q17A', 'Q21A', 'Q24A', 'Q26A', 'Q31A', 'Q34A', 'Q37A', 'Q38A', 'Q42A']
df_filtered[item_columns] = df_filtered[item_columns] - 1


df_filtered['soma_depressao'] = df_filtered[item_columns].sum(axis=1)

# Função para mapear o índice de depressão
def mapear_indice_depressao(pontuacao):
    if pontuacao <= 9:
        return 0
    elif pontuacao <= 13:
        return 1
    elif pontuacao <= 20:
        return 2
    elif pontuacao <= 27:
        return 3
    else:
        return 4

# indice depressoa
df_filtered['indice_depressao'] = df_filtered['soma_depressao'].apply(mapear_indice_depressao)


country_legend = {number: country for number, country in enumerate(df_filtered['country'].unique(), 1)}
print("Legenda para a coluna 'country':")
for number, country in country_legend.items():
    print(f"{number}: {country}")

# Remover a coluna 'soma_depressao'
df_filtered.drop(columns=['soma_depressao'], inplace=True)

# Salvar o dataset filtrado
df_filtered.to_csv('dataset.csv', index=False)

# Número final de linhas
final_row_count = len(df_filtered)
print(f"Número de linhas finais no dataset filtrado: {final_row_count}")


Número inicial de linhas no originalDataset: 39775
Número de linhas após todos os filtros: 24927
Legenda para a coluna 'country':
1: MY
2: GB
3: DE
4: US
5: AU
6: CA
7: FI
8: ES
9: ID
10: GR
11: IN
12: BR
13: PT
14: RS
15: IT
16: FR
17: RO
18: PL
19: PH
20: IE
21: NZ
22: ZA
23: NL
24: JM
25: MX
26: PK
27: HK
28: SG
29: RU
30: TR
31: EG
32: SA
33: BN
34: JP
Número de linhas finais no dataset filtrado: 24927


In [7]:
import pandas as pd


df = pd.read_csv('dataset.csv')
# encontra  colunas categorizáveis
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()


print(f"Colunas categorizáveis identificadas para One-Hot Encoding: {categorical_columns}")


categorical_columns = [col for col in categorical_columns if col != 'indice_depressao']

# One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# adicionar 'indice_depressao' de volta
df_encoded['indice_depressao'] = df['indice_depressao']


df_encoded.to_csv('dataset_one_hot_encoded.csv', index=False)

print(f"Número de linhas no dataset codificado: {len(df_encoded)}")
print(f"Número de colunas no dataset codificado: {len(df_encoded.columns)}")


Colunas categorizáveis identificadas para One-Hot Encoding: ['country']
Número de linhas no dataset codificado: 24927
Número de colunas no dataset codificado: 53


In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
import json


data = pd.read_csv('dataset_one_hot_encoded.csv')

# x e y 
X = data.drop(columns=['indice_depressao'])
y = data['indice_depressao']

# 80% treino, 20% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# normalização
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

#  SMOTE
smote = SMOTE(random_state=42)
X_train_scaled, y_train = smote.fit_resample(X_train_scaled, y_train)


modelos = {
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Regressão Logística': LogisticRegression(),
    'K-Vizinhos Mais Próximos': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Árvore de Decisão': DecisionTreeClassifier(),
    'AdaBoost': AdaBoostClassifier()
}

# Validação cruzada
resultados = {}
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for nome, modelo in modelos.items():
    dados_fold = []
    for train_idx, test_idx in skf.split(X_train_scaled, y_train):
        X_train_fold, X_test_fold = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]

        # treinamento e predição
        modelo.fit(X_train_fold, y_train_fold)
        y_pred = modelo.predict(X_test_fold)

        # garantir que y_pred seja um vetor 1D
        if len(y_pred.shape) > 1:
            y_pred = y_pred.ravel()


        accuracy = accuracy_score(y_test_fold, y_pred)
        erro_percentual = (1 - accuracy) * 100
        tamanho_subdataset = len(y_test_fold)
        erros = (y_test_fold != y_pred).sum()

        # resultados do fold
        dados_fold.append({
            "Subdataset": f"fold {len(dados_fold)}",
            "Erro em %": round(erro_percentual, 3),
            "Tamanho do Subdataset": tamanho_subdataset,
            "Acurácia em %": round(accuracy * 100, 3),
            "Erros do Algoritmo": erros
        })

    #  acurácias
    media_acuracia = np.mean([fold["Acurácia em %"] for fold in dados_fold])

    # armazenar resultados do modelo no formato requerido
    resultados[nome] = round(media_acuracia / 100, 4)  # armazena a acurácia média como decimal

    # Impressão dos resultados no terminal
    print(f"\nResultados para {nome}:")
    print(pd.DataFrame(dados_fold))
    print(f"Acurácia média para {nome}: {round(media_acuracia, 3)}%")

# Salvar os resultados em formato JSON
with open('metricas_modelos.json', 'w') as f:
    json.dump(resultados, f, indent=4)

print("\nOs resultados foram salvos no arquivo 'metricas_modelos.json'.")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(



Resultados para Random Forest:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0      4.466                   2933         95.534   
1     fold 1      3.989                   2933         96.011   
2     fold 2      4.603                   2933         95.397   
3     fold 3      3.853                   2933         96.147   
4     fold 4      3.853                   2933         96.147   
5     fold 5      3.513                   2932         96.487   
6     fold 6      4.297                   2932         95.703   
7     fold 7      4.297                   2932         95.703   
8     fold 8      4.263                   2932         95.737   
9     fold 9      4.434                   2932         95.566   

   Erros do Algoritmo  
0                 131  
1                 117  
2                 135  
3                 113  
4                 113  
5                 103  
6                 126  
7                 126  
8                 125  
9           

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c


Resultados para Regressão Logística:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0      0.341                   2933         99.659   
1     fold 1      0.239                   2933         99.761   
2     fold 2      0.170                   2933         99.830   
3     fold 3      0.341                   2933         99.659   
4     fold 4      0.273                   2933         99.727   
5     fold 5      0.273                   2932         99.727   
6     fold 6      0.273                   2932         99.727   
7     fold 7      0.409                   2932         99.591   
8     fold 8      0.409                   2932         99.591   
9     fold 9      0.409                   2932         99.591   

   Erros do Algoritmo  
0                  10  
1                   7  
2                   5  
3                  10  
4                   8  
5                   8  
6                   8  
7                  12  
8                  12  
9     

In [ ]:
Resultados para Random Forest:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0      4.466                   2933         95.534
1     fold 1      3.989                   2933         96.011
2     fold 2      4.603                   2933         95.397
3     fold 3      3.853                   2933         96.147
4     fold 4      3.853                   2933         96.147
5     fold 5      3.513                   2932         96.487
6     fold 6      4.297                   2932         95.703
7     fold 7      4.297                   2932         95.703
8     fold 8      4.263                   2932         95.737
9     fold 9      4.434                   2932         95.566

   Erros do Algoritmo
0                 131
1                 117
2                 135
3                 113
4                 113
5                 103
6                 126
7                 126
8                 125
9                 130
Acurácia média para Random Forest: 95.843%

Resultados para SVM:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0      0.375                   2933         99.625
1     fold 1      0.239                   2933         99.761
2     fold 2      0.307                   2933         99.693
3     fold 3      0.580                   2933         99.420
4     fold 4      0.170                   2933         99.830
5     fold 5      0.136                   2932         99.864
6     fold 6      0.375                   2932         99.625
7     fold 7      0.443                   2932         99.557
8     fold 8      0.443                   2932         99.557
9     fold 9      0.341                   2932         99.659

   Erros do Algoritmo
0                  11
1                   7
2                   9
3                  17
4                   5
5                   4
6                  11
7                  13
8                  13
9                  10
Acurácia média para SVM: 99.659%

Resultados para Regressão Logística:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0      0.341                   2933         99.659
1     fold 1      0.239                   2933         99.761
2     fold 2      0.170                   2933         99.830
3     fold 3      0.341                   2933         99.659
4     fold 4      0.273                   2933         99.727
5     fold 5      0.273                   2932         99.727
6     fold 6      0.273                   2932         99.727
7     fold 7      0.409                   2932         99.591
8     fold 8      0.409                   2932         99.591
9     fold 9      0.409                   2932         99.591

   Erros do Algoritmo
0                  10
1                   7
2                   5
3                  10
4                   8
5                   8
6                   8
7                  12
8                  12
9                  12
Acurácia média para Regressão Logística: 99.686%

Resultados para K-Vizinhos Mais Próximos:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0      8.149                   2933         91.851
1     fold 1      8.694                   2933         91.306
2     fold 2      7.740                   2933         92.260
3     fold 3      9.206                   2933         90.794
4     fold 4      7.705                   2933         92.295
5     fold 5      7.776                   2932         92.224
6     fold 6      8.015                   2932         91.985
7     fold 7      8.015                   2932         91.985
8     fold 8      8.049                   2932         91.951
9     fold 9      7.913                   2932         92.087

   Erros do Algoritmo
0                 239
1                 255
2                 227
3                 270
4                 226
5                 228
6                 235
7                 235
8                 236
9                 232
Acurácia média para K-Vizinhos Mais Próximos: 91.874%

Resultados para Naive Bayes:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0     33.413                   2933         66.587
1     fold 1     30.140                   2933         69.860
2     fold 2     32.867                   2933         67.133
3     fold 3     33.890                   2933         66.110
4     fold 4     32.185                   2933         67.815
5     fold 5     31.003                   2932         68.997
6     fold 6     28.070                   2932         71.930
7     fold 7     29.945                   2932         70.055
8     fold 8     31.583                   2932         68.417
9     fold 9     31.548                   2932         68.452

   Erros do Algoritmo
0                 980
1                 884
2                 964
3                 994
4                 944
5                 909
6                 823
7                 878
8                 926
9                 925
Acurácia média para Naive Bayes: 68.536%

Resultados para Árvore de Decisão:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0     15.513                   2933         84.487
1     fold 1     15.581                   2933         84.419
2     fold 2     15.172                   2933         84.828
3     fold 3     15.479                   2933         84.521
4     fold 4     16.331                   2933         83.669
5     fold 5     14.973                   2932         85.027
6     fold 6     15.587                   2932         84.413
7     fold 7     15.791                   2932         84.209
8     fold 8     13.950                   2932         86.050
9     fold 9     16.098                   2932         83.902

   Erros do Algoritmo
0                 455
1                 457
2                 445
3                 454
4                 479
5                 439
6                 457
7                 463
8                 409
9                 472
Acurácia média para Árvore de Decisão: 84.552%

Resultados para AdaBoost:
  Subdataset  Erro em %  Tamanho do Subdataset  Acurácia em %  \
0     fold 0     52.506                   2933         47.494
1     fold 1     54.859                   2933         45.141
2     fold 2     45.892                   2933         54.108
3     fold 3     51.790                   2933         48.210
4     fold 4     49.676                   2933         50.324
5     fold 5     57.060                   2932         42.940
6     fold 6     49.761                   2932         50.239
7     fold 7     50.409                   2932         49.591
8     fold 8     41.337                   2932         58.663
9     fold 9     53.070                   2932         46.930

   Erros do Algoritmo
0                1540
1                1609
2                1346
3                1519
4                1457
5                1673
6                1459
7                1478
8                1212
9                1556
Acurácia média para AdaBoost: 49.364%

In [10]:
import json

# Carregar o arquivo JSON
with open('metricas_modelos.json', 'r') as f:
    metricas_modelos = json.load(f)

# Exibir as métricas no console
print("Resultados das Acurácias Médias:")
for modelo, acuracia in metricas_modelos.items():
    print(f"{modelo}: {acuracia:.4f}")


Resultados das Acurácias Médias:
Random Forest: 0.9584
SVM: 0.9966
Regressão Logística: 0.9969
K-Vizinhos Mais Próximos: 0.9187
Naive Bayes: 0.6854
Árvore de Decisão: 0.8455
AdaBoost: 0.4936


In [11]:
import pandas as pd

data = pd.read_csv('dataset.csv')

# mapear iso
country_name_dict = {
    'AU': 'Australia', 'BN': 'Brunei', 'BR': 'Brazil', 'CA': 'Canada', 'DE': 'Germany', 'EG': 'Egypt',
    'ES': 'Spain', 'FI': 'Finland', 'FR': 'France', 'GB': 'United Kingdom', 'GR': 'Greece', 'HK': 'Hong Kong',
    'ID': 'Indonesia', 'IE': 'Ireland', 'IN': 'India', 'IT': 'Italy', 'JM': 'Jamaica', 'JP': 'Japan',
    'MX': 'Mexico', 'MY': 'Malaysia', 'NL': 'Netherlands', 'NZ': 'New Zealand', 'PH': 'Philippines',
    'PK': 'Pakistan', 'PL': 'Poland', 'PT': 'Portugal', 'RO': 'Romania', 'RS': 'Serbia', 'RU': 'Russia',
    'SA': 'Saudi Arabia', 'SG': 'Singapore', 'TR': 'Turkey', 'US': 'United States', 'ZA': 'South Africa'
}

# mapear os países para seus respectivos continentes
continent_dict = {
    'AU': 'Australia/New Zealand', 'BN': 'Asia', 'BR': 'South America', 'CA': 'North America', 'DE': 'Europe',
'EG': 'Africa', 'ES': 'Europe', 'FI': 'Europe', 'FR': 'Europe', 'GB': 'Europe', 'GR': 'Europe',
    'HK': 'Asia', 'ID': 'Asia', 'IE': 'Europe', 'IN': 'Asia', 'IT': 'Europe', 'JM': 'North America',
        'JP': 'Asia', 'MX': 'North America', 'MY': 'Asia', 'NL': 'Europe', 'NZ': 'Australia/New Zealand',
    'PH': 'Asia', 'PK': 'Asia', 'PL': 'Europe', 'PT': 'Europe', 'RO': 'Europe', 'RS': 'Europe',
    'RU': 'Europe', 'SA': 'Asia', 'SG': 'Asia', 'TR': 'Asia', 'US': 'North America', 'ZA': 'Africa'
}

# testa se as colunas country e indice_depressao estão no dataset
if 'country' in data.columns and 'indice_depressao' in data.columns:

    data['continent'] = data['country'].map(continent_dict)


    print("Exemplo de mapeamento de países para continentes:")
    print(data[['country', 'continent']].head())

    # contagem /continente
    continent_counts = data['continent'].value_counts()

    for continent in continent_counts.index:
        print(f"\nContinente: {continent}, Total de participantes: {continent_counts[continent]}")
        countries_in_continent = data[data['continent'] == continent]['country'].value_counts()

        continent_avg_depression = 0
        country_count = 0

        for country_code, count in countries_in_continent.items():
            avg_depression = data[data['country'] == country_code]['indice_depressao'].mean()
            country_name = country_name_dict[country_code]  # Obter o nome do país
            print(f"  País: {country_name}, Participantes: {count}, Média do Índice de Depressão: {avg_depression:.2f}")

            # somar para calcular a média do continente
            continent_avg_depression += avg_depression
            country_count += 1

        continent_avg_depression /= country_count
        print(f"Média do Índice de Depressão no Continente ({continent}): {continent_avg_depression:.2f}")

    # exibir a soma total de todos os participantes
    total_participants = continent_counts.sum()
    print(f"\nTotal de participantes em todos os continentes: {total_participants}")

else:
    print("As colunas 'country' ou 'indice_depressao' não foram encontradas no dataset.")


Exemplo de mapeamento de países para continentes:
  country      continent
0      MY           Asia
1      GB         Europe
2      DE         Europe
3      US  North America
4      US  North America

Continente: Asia, Total de participantes: 19180
  País: Malaysia, Participantes: 17252, Média do Índice de Depressão: 2.12
  País: Indonesia, Participantes: 642, Média do Índice de Depressão: 2.32
  País: Philippines, Participantes: 472, Média do Índice de Depressão: 2.46
  País: India, Participantes: 331, Média do Índice de Depressão: 2.23
  País: Singapore, Participantes: 123, Média do Índice de Depressão: 2.08
  País: Brunei, Participantes: 97, Média do Índice de Depressão: 2.18
  País: Japan, Participantes: 74, Média do Índice de Depressão: 1.84
  País: Turkey, Participantes: 54, Média do Índice de Depressão: 2.39
  País: Pakistan, Participantes: 53, Média do Índice de Depressão: 2.32
  País: Saudi Arabia, Participantes: 43, Média do Índice de Depressão: 1.91
  País: Hong Kong, Partic

In [ ]:
Exemplo de mapeamento de países para continentes:
  country      continent
0      MY           Asia
1      GB         Europe
2      DE         Europe
3      US  North America
4      US  North America

Continente: Asia, Total de participantes: 19180
  País: Malaysia, Participantes: 17252, Média do Índice de Depressão: 2.12
  País: Indonesia, Participantes: 642, Média do Índice de Depressão: 2.32
  País: Philippines, Participantes: 472, Média do Índice de Depressão: 2.46
  País: India, Participantes: 331, Média do Índice de Depressão: 2.23
  País: Singapore, Participantes: 123, Média do Índice de Depressão: 2.08
  País: Brunei, Participantes: 97, Média do Índice de Depressão: 2.18
  País: Japan, Participantes: 74, Média do Índice de Depressão: 1.84
  País: Turkey, Participantes: 54, Média do Índice de Depressão: 2.39
  País: Pakistan, Participantes: 53, Média do Índice de Depressão: 2.32
  País: Saudi Arabia, Participantes: 43, Média do Índice de Depressão: 1.91
  País: Hong Kong, Participantes: 39, Média do Índice de Depressão: 2.13
Média do Índice de Depressão no Continente (Asia): 2.18

Continente: North America, Total de participantes: 3813
  País: United States, Participantes: 3277, Média do Índice de Depressão: 2.09
  País: Canada, Participantes: 428, Média do Índice de Depressão: 2.12
  País: Mexico, Participantes: 75, Média do Índice de Depressão: 2.25
  País: Jamaica, Participantes: 33, Média do Índice de Depressão: 2.24
Média do Índice de Depressão no Continente (North America): 2.18

Continente: Europe, Total de participantes: 1322
  País: United Kingdom, Participantes: 517, Média do Índice de Depressão: 2.16
  País: Germany, Participantes: 134, Média do Índice de Depressão: 2.26
  País: France, Participantes: 125, Média do Índice de Depressão: 2.28
  País: Poland, Participantes: 85, Média do Índice de Depressão: 2.69
  País: Italy, Participantes: 72, Média do Índice de Depressão: 2.50
  País: Romania, Participantes: 71, Média do Índice de Depressão: 2.01
  País: Netherlands, Participantes: 59, Média do Índice de Depressão: 2.24
  País: Spain, Participantes: 43, Média do Índice de Depressão: 1.81
  País: Ireland, Participantes: 39, Média do Índice de Depressão: 2.26
  País: Russia, Participantes: 39, Média do Índice de Depressão: 2.31
  País: Greece, Participantes: 37, Média do Índice de Depressão: 2.03
  País: Serbia, Participantes: 35, Média do Índice de Depressão: 1.46
  País: Portugal, Participantes: 34, Média do Índice de Depressão: 2.12
  País: Finland, Participantes: 32, Média do Índice de Depressão: 2.34
Média do Índice de Depressão no Continente (Europe): 2.18

Continente: Australia/New Zealand, Total de participantes: 400
  País: Australia, Participantes: 278, Média do Índice de Depressão: 2.11
  País: New Zealand, Participantes: 122, Média do Índice de Depressão: 2.21
Média do Índice de Depressão no Continente (Australia/New Zealand): 2.16

Continente: Africa, Total de participantes: 107
  País: Egypt, Participantes: 55, Média do Índice de Depressão: 2.58
  País: South Africa, Participantes: 52, Média do Índice de Depressão: 2.13
Média do Índice de Depressão no Continente (Africa): 2.36

Continente: South America, Total de participantes: 105
  País: Brazil, Participantes: 105, Média do Índice de Depressão: 2.20
Média do Índice de Depressão no Continente (South America): 2.20

Total de participantes em todos os continentes: 24927

In [ ]:
# nome no GeoJSON
country_names_from_geojson = [feature['properties']['name'] for feature in geojson_data['features']]
print(country_names_from_geojson)



['Afghanistan', 'Angola', 'Albania', 'United Arab Emirates', 'Argentina', 'Armenia', 'Antarctica', 'French Southern and Antarctic Lands', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'The Bahamas', 'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Brunei', 'Bhutan', 'Botswana', 'Central African Republic', 'Canada', 'Switzerland', 'Chile', 'China', 'Ivory Coast', 'Cameroon', 'Democratic Republic of the Congo', 'Republic of the Congo', 'Colombia', 'Costa Rica', 'Cuba', 'Northern Cyprus', 'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt', 'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji', 'Falkland Islands', 'France', 'Gabon', 'United Kingdom', 'Georgia', 'Ghana', 'Guinea', 'Gambia', 'Guinea Bissau', 'Equatorial Guinea', 'Greece', 'Greenland', 'Guatemala', 'French Guiana', 'Guyana', 'Honduras', 'Croatia', 'Haiti', 'Hunga

In [ ]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.23.200.150


In [ ]:
!npm install -g localtunnel@2.0.2

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
changed 22 packages in 2s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.200.150:8501

⠼⠴⠦your url is: https://shiny-otters-bake.loca.lt
2024-12-31 23:43:26.165 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-12-31 23:43:35.112 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-12-31 23:43:59.101 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
  Stopping...
^C
